In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

The Python Record Linkage Toolkit is a library to link records in or between data sources. The toolkit provides most of the tools needed for record linkage and deduplication. The package contains indexing methods, functions to compare records and classifiers. 

# Link two datasets using the recordlinkage Python package

https://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html

This example shows how two datasets with data about persons can be linked. We will try to link the data based on attributes like first name, surname, sex, date of birth, place and address.

In [2]:
#For this example, we use the Febrl datasets 4A and 4B. These datasets can be loaded with the function load_febrl4
import recordlinkage
from recordlinkage.datasets import load_febrl4

In [5]:
dfA, dfB = load_febrl4()
dfA.head()
dfB.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


In [7]:
dfA.shape
dfB.shape

(5000, 10)

(5000, 10)

In [8]:
# We want to make record pairs. This is also called indexing.
# First, load the index.Index class and call the .full method.
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)

<Index>

In [9]:
#With the method index, all possible (and unique) record pairs are made. The method returns a pandas.MultiIndex.
print (len(dfA), len(dfB), len(pairs))

5000 5000 25000000


In [10]:
#One of the most well known indexing methods is named blocking. 
#This method includes only record pairs that are identical on one or more stored attributes of the person.
indexer = recordlinkage.Index()
indexer.block('given_name')
#The argument ‘given_name’ is the blocking variable. 
candidate_links = indexer.index(dfA, dfB)

print (len(candidate_links))

<Index>

77249


In [11]:
#Each record pair is a candidate match. 
#To classify the candidate record pairs into matches and non-matches, 
#compare the records on all attributes both records have in common. 
#The recordlinkage module has a class named Compare. This class is used to compare the records.


# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA, dfB)
#All attribute comparisons are stored in a DataFrame with horizontally the features and vertically the record pairs.

<Compare>

<Compare>

<Compare>

<Compare>

<Compare>

<Compare>

In [12]:
features

given_name  surname  date_of_birth  suburb  \
rec_id_1     rec_id_2                                                     
rec-1070-org rec-3024-dup-0           1      0.0              0       0   
             rec-2371-dup-0           1      0.0              0       0   
             rec-4652-dup-0           1      0.0              0       0   
             rec-4795-dup-0           1      0.0              0       0   
             rec-1314-dup-0           1      0.0              0       0   
rec-2371-org rec-3024-dup-0           1      0.0              0       0   
             rec-2371-dup-0           1      1.0              1       1   
             rec-4652-dup-0           1      0.0              0       0   
             rec-4795-dup-0           1      0.0              0       0   
             rec-1314-dup-0           1      0.0              0       0   
rec-3582-org rec-3024-dup-0           1      0.0              0       0   
             rec-2371-dup-0           1      0.0              0       0   
             rec-4652-dup-0           1      0.0              0       0   
             rec-4795-dup-0           1      0.0              0       0   
             rec-1314-dup-0           1      0.0              0       0   
rec-3024-org rec-3024-dup-0           1      1.0              1       0   
             rec-2371-dup-0           1      0.0              0       0   
             rec-4652-dup-0           1      0.0              0       0   
             rec-4795-dup-0           1      0.0              0       0   
             rec-1314-dup-0           1      0.0              0       0   
rec-4652-org rec-3024-dup-0           1      0.0              0       0   
             rec-2371-dup-0           1      0.0              0       0   
             rec-4652-dup-0           1      1.0              1       0   
             rec-4795-dup-0           1      0.0              0       0   
             rec-1314-dup-0           1      0.0              0       0   
rec-4795-org rec-3024-dup-0           1      0.0              0       0   
             rec-2371-dup-0           1      0.0              0       0   
             rec-4652-dup-0           1      0.0              0       0   
             rec-4795-dup-0           1      1.0              1       1   
             rec-1314-dup-0           1      0.0              0       0   
...                                 ...      ...            ...     ...   
rec-2820-org rec-2820-dup-0           1      1.0              1       1   
             rec-991-dup-0            1      0.0              0       0   
rec-1984-org rec-1984-dup-0           1      1.0              1       1   
rec-1662-org rec-1984-dup-0           1      0.0              0       0   
rec-4415-org rec-1984-dup-0           1      0.0              0       0   
rec-1920-org rec-1920-dup-0           1      1.0              1       1   
rec-303-org  rec-303-dup-0            1      1.0              1       1   
rec-1915-org rec-1915-dup-0           1      1.0              1       1   
rec-4739-org rec-4739-dup-0           1      1.0              1       0   
             rec-4865-dup-0           1      0.0              0       0   
rec-681-org  rec-4276-dup-0           1      0.0              0       0   
rec-4603-org rec-4848-dup-0           1      0.0              0       0   
             rec-4603-dup-0           1      1.0              1       1   
rec-3122-org rec-4848-dup-0           1      0.0              0       0   
             rec-4603-dup-0           1      0.0              0       0   
rec-3711-org rec-3711-dup-0           1      1.0              1       1   
rec-4912-org rec-4912-dup-0           1      1.0              1       1   
rec-664-org  rec-664-dup-0            1      1.0              1       1   
             rec-1311-dup-0           1      0.0              0       0   
rec-4031-org rec-4031-dup-0           1      1.0              1       1   
rec-1413-org rec-1413-dup-0           1      1.0      

In [13]:
features.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,77249.0,77249.000000,77249.000000,77249.000000,77249.000000,77249.000000
mean,1.0,0.044428,0.037929,0.032259,0.248767,0.036700
std,0.0,0.206045,0.191027,0.176689,0.432301,0.188024
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
#The last step is to decide which records belong to the same person.
# Sum the comparison results for each pair
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0     1566
5.0     1332
4.0      343
3.0      146
2.0    16427
1.0    57435
dtype: int64

In [15]:
features[features.sum(axis=1) > 3]

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-2371-org,rec-2371-dup-0,1,1.0,1,1,1,1.0
rec-3024-org,rec-3024-dup-0,1,1.0,1,0,1,0.0
rec-4652-org,rec-4652-dup-0,1,1.0,1,0,1,1.0
rec-4795-org,rec-4795-dup-0,1,1.0,1,1,1,1.0
rec-1016-org,rec-1016-dup-0,1,1.0,1,1,0,1.0
rec-2463-org,rec-2463-dup-0,1,1.0,0,1,1,1.0
rec-521-org,rec-521-dup-0,1,1.0,1,1,1,1.0
rec-786-org,rec-786-dup-0,1,1.0,1,1,1,1.0
rec-572-org,rec-572-dup-0,1,0.0,0,1,1,1.0


# Data deduplication using recordlinkage Python package

In [16]:
#The submodule recordlinkage.datasets contains several datasets that can be used for testing. 
#For this example, we use the Febrl dataset 1. 
#This dataset contains 1000 records of which 500 original and 500 duplicates, with exactly one duplicate per original record. 

import recordlinkage
from recordlinkage.datasets import load_febrl1
dfA = load_febrl1()
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


In [17]:
#Make record pairs
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(dfA)

<Index>

In [18]:
print (len(dfA), len(candidate_links))
# (1000*1000-1000)/2 = 499500

1000 499500


In [19]:
#Many of these record pairs do not belong to the same person.
#Let us do blocking
indexer = recordlinkage.Index()
indexer.block('given_name')
candidate_links = indexer.index(dfA)

print (len(candidate_links))

<Index>

2082


In [21]:
#Compare records
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA)

<Compare>

<Compare>

<Compare>

<Compare>

<Compare>

<Compare>

In [22]:
features.head(10)
features.describe()

given_name  surname  date_of_birth  suburb  \
rec_id_1      rec_id_2                                                    
rec-183-dup-0 rec-122-org             1      0.0              0       0   
rec-248-org   rec-122-org             1      0.0              0       0   
              rec-183-dup-0           1      0.0              0       0   
rec-122-dup-0 rec-122-org             1      1.0              1       1   
              rec-183-dup-0           1      0.0              0       0   
              rec-248-org             1      0.0              0       0   
rec-469-org   rec-122-org             1      0.0              0       0   
              rec-183-dup-0           1      0.0              0       0   
              rec-248-org             1      0.0              0       0   
              rec-122-dup-0           1      0.0              0       0   

                             state  address_1  
rec_id_1      rec_id_2                         
rec-183-dup-0 rec-122-org        0        0.0  
rec-248-org   rec-122-org        1        0.0  
              rec-183-dup-0      0        0.0  
rec-122-dup-0 rec-122-org        1        1.0  
              rec-183-dup-0      0        0.0  
              rec-248-org        1        0.0  
rec-469-org   rec-122-org        0        0.0  
              rec-183-dup-0      1        0.0  
              rec-248-org        0        0.0  
              rec-122-dup-0      0        0.0

,given_name,surname,date_of_birth,suburb,state,address_1
count,2082.0,2082.000000,2082.000000,2082.000000,2082.000000,2082.000000
mean,1.0,0.144092,0.139289,0.108549,0.327089,0.133045
std,0.0,0.351268,0.346331,0.311148,0.469263,0.339705
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)
matches = features[features.sum(axis=1) > 3]

print(len(matches))
matches.head(10)

6.0     142
5.0     145
4.0      30
3.0       9
2.0     376
1.0    1380
dtype: int64

317


,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-122-dup-0,rec-122-org,1,1.0,1,1,1,1.0
rec-183-org,rec-183-dup-0,1,1.0,1,1,1,1.0
rec-248-dup-0,rec-248-org,1,1.0,1,1,1,1.0
rec-373-dup-0,rec-373-org,1,1.0,1,1,1,1.0
rec-10-org,rec-10-dup-0,1,1.0,1,1,1,1.0
rec-342-dup-0,rec-342-org,1,1.0,0,1,1,1.0
rec-397-org,rec-397-dup-0,1,1.0,1,1,1,0.0
rec-472-org,rec-472-dup-0,1,1.0,1,1,1,0.0
rec-330-org,rec-330-dup-0,1,0.0,1,1,1,0.0


# Classification Algorithms for Record Linkage

In [24]:
#In the context of record linkage, classification refers to the process of dividing record pairs 
#into matches and non-matches (distinct pairs). 

The examples below make use of the Krebs register (German for cancer registry) dataset. The Krebs register dataset contains comparison vectors of a large set of record pairs. For each record pair, it is known if the records represent the same person (match) or not (non-match). 

The dataset contains 5749132 compared record pairs and has the following variables: first name, last name, sex, birthday, birth month, birth year and zip code. The Krebs register contains len(krebs_true_links) == 20931 matching record pairs.

In [25]:
import recordlinkage as rl
from recordlinkage.datasets import load_krebsregister

krebs_X, krebs_true_links = load_krebsregister(missing_values=0)
krebs_X

Start downloading the data.
Data download succesfull.


,,cmp_firstname1,cmp_firstname2,cmp_lastname1,cmp_lastname2,cmp_sex,cmp_birthday,cmp_birthmonth,cmp_birthyear,cmp_zipcode
id1,id2,,,,,,,,,
22161,38467,1.000000,0.0,0.142857,0.0,1,0.0,1.0,0.0,0.0
38713,75352,0.000000,0.0,0.571429,0.0,1,0.0,0.0,0.0,0.0
13699,32825,0.166667,0.0,0.000000,0.0,0,1.0,1.0,1.0,0.0
22709,37682,0.285714,0.0,1.000000,0.0,1,0.0,0.0,0.0,0.0
2342,69060,0.250000,0.0,0.125000,0.0,1,1.0,1.0,1.0,0.0
49596,92145,1.000000,0.0,0.000000,0.0,1,0.0,1.0,0.0,0.0
13092,95695,1.000000,0.0,0.142857,0.0,1,0.0,0.0,1.0,0.0
1151,45872,0.400000,0.0,0.555556,0.0,1,0.0,1.0,0.0,0.0
1732,38913,0.444444,0.0,1.000000,0.0,1,0.0,1.0,0.0,1.0


In [26]:
krebs_X.describe().T

,count,mean,std,min,25%,50%,75%,max
cmp_firstname1,5749132.0,0.712778,0.388839,0.0,0.285714,1.000000,1.000000,1.0
cmp_firstname2,5749132.0,0.016234,0.125199,0.0,0.000000,0.000000,0.000000,1.0
cmp_lastname1,5749132.0,0.315628,0.334234,0.0,0.100000,0.181818,0.428571,1.0
cmp_lastname2,5749132.0,0.000136,0.010081,0.0,0.000000,0.000000,0.000000,1.0
cmp_sex,5749132.0,0.955001,0.207301,0.0,1.000000,1.000000,1.000000,1.0
cmp_birthday,5749132.0,0.224434,0.417209,0.0,0.000000,0.000000,0.000000,1.0
cmp_birthmonth,5749132.0,0.488788,0.499874,0.0,0.000000,0.000000,1.000000,1.0
cmp_birthyear,5749132.0,0.222718,0.416070,0.0,0.000000,0.000000,0.000000,1.0
cmp_zipcode,5749132.0,0.005516,0.074067,0.0,0.000000,0.000000,0.000000,1.0


In [36]:
# We use the true match status of the first 5000 record pairs of the Krebs register data as training data.
golden_pairs = krebs_X[0:5000]
golden_matches_index = golden_pairs.index & krebs_true_links # 2093 matching pairs

In [30]:
#Using logistic regression classifier
# Initialize the classifier
logreg = rl.LogisticRegressionClassifier()

# Train the classifier
logreg.fit(golden_pairs, golden_matches_index)
print ("Intercept: ", logreg.intercept)
print ("Coefficients: ", logreg.coefficients)

Intercept:  -6.298043571006424
Coefficients:  [ 4.90452843e-01  1.21640484e-01  2.15040485e+00 -2.84818101e-03
 -1.79712465e+00  9.61085558e-01  6.72610441e-02  1.03408608e+00
  4.30556110e+00]


/root/anaconda3/envs/py36/lib/python3.6/site-packages/recordlinkage/base.py:949: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self._fit(comparison_vectors.as_matrix(), y.values)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [38]:
# Predict the match status for all record pairs
result_logreg = logreg.predict(krebs_X)
len(result_logreg)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/recordlinkage/base.py:1020: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  prediction = self._predict(comparison_vectors.as_matrix())


20150

In [39]:
#Logistic regression identified 20150 matches (c)
rl.confusion_matrix(krebs_true_links, result_logreg, len(krebs_X))

array([[  19884,    1047],
       [    266, 5727935]])

In [40]:
# The F-score for this prediction is
rl.fscore(krebs_true_links, result_logreg)

0.9680387527080645

The predicted number of matches is not much more than the 20931 true matches. The result was achieved with a small training dataset of 5000 record pairs.

In [41]:
#One can also provide user-defined weights for various features without any learning
intercept = -9
coefficients = [2.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0]

logreg = rl.LogisticRegressionClassifier(coefficients, intercept)

# predict without calling LogisticRegressionClassifier.fit
result_logreg_pretrained = logreg.predict(krebs_X)
print (len(result_logreg_pretrained))

/root/anaconda3/envs/py36/lib/python3.6/site-packages/recordlinkage/base.py:1020: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  prediction = self._predict(comparison_vectors.as_matrix())


21303


In [42]:
rl.confusion_matrix(krebs_true_links, result_logreg_pretrained, len(krebs_X))
# The F-score for this classification is
rl.fscore(krebs_true_links, result_logreg_pretrained)

array([[  20857,      74],
       [    446, 5727755]])

0.987687645025335

Well, if you increase data for training logistic regression, it will also lead to such good results :-)

In [44]:
#Naive Bayes classifier
# Train the classifier
nb = rl.NaiveBayesClassifier(binarize=0.3)
nb.fit(golden_pairs, golden_matches_index)

# Predict the match status for all record pairs
result_nb = nb.predict(krebs_X)

len(result_nb)
rl.confusion_matrix(krebs_true_links, result_nb, len(krebs_X))
# The F-score for this classification is
rl.fscore(krebs_true_links, result_nb)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/recordlinkage/base.py:949: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self._fit(comparison_vectors.as_matrix(), y.values)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/recordlinkage/base.py:1020: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  prediction = self._predict(comparison_vectors.as_matrix())


19837

array([[  19825,    1106],
       [     12, 5728189]])

0.9725765306122448

In [45]:
#SVM
# Train the classifier
svm = rl.SVMClassifier()
svm.fit(golden_pairs, golden_matches_index)

# Predict the match status for all record pairs
result_svm = svm.predict(krebs_X)

len(result_svm)
rl.confusion_matrix(krebs_true_links, result_svm, len(krebs_X))
# The F-score for this classification is
rl.fscore(krebs_true_links, result_svm)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/recordlinkage/base.py:949: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self._fit(comparison_vectors.as_matrix(), y.values)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/recordlinkage/base.py:1020: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  prediction = self._predict(comparison_vectors.as_matrix())


20839

array([[  20825,     106],
       [     14, 5728187]])

0.997127124730668

# Using the dedupe package in Python
Works only with Linux and Python 2.x

git clone https://github.com/DistrictDataLabs/dedupe-examples.git

cd dedupe-examples

pip install unidecode

pip install future

pip install dedupe


python csv_example.py

python csv_evaluation.py

More examples here: https://github.com/dedupeio/dedupe-examples